In [1]:
import keys
import requests
from requests_oauthlib import OAuth1
import json

auth = OAuth1(keys.api_key, keys.api_secret_key, keys.access_token, keys.access_token_secret)

def dump_json(data, filename):
    with open(filename, 'w') as outfile:
        json.dump(data, outfile)

def request_data(url_req, params, auth):
    data = []
    fromDate = None
    toDate = None
    
    while True:
        results = requests.get(url_rest, params=params, auth=auth)
        is_response_ok = results.ok
        if is_response_ok:
            results = results.json()
            data = data + results['results']
            if 'next' in results.keys():
                params['next'] = results['next']
            else:
                fromDate = results['requestParameters']['fromDate']
                toDate = results['requestParameters']['toDate']
                break
    return {'results':data, 'requestParameters':{'fromDate':fromDate, 'toDate':toDate}}

if __name__ == '__main__':
    
    url_rest = "https://api.twitter.com/1.1/tweets/search/fullarchive/testFull.json"
    query = 'from:midasIIITD'
    fromDate = 201809010000
    params = {'query': query, 'maxResults':100, 'next':None, 'fromDate':fromDate}

    data = request_data(url_rest, params, auth)
    
    out_filename = 'data.json'
    dump_json(data, out_filename)

KeyboardInterrupt: 

In [1]:
import json
from collections import defaultdict
import pandas as pd
from texttable import Texttable

def load_json(filename):
    with open(filename, 'r') as infile:
        data = json.load(infile)
    return data

def get_num_images(tweet_data):
    num_images = 0
    if 'extended_tweet' in tweet_data.keys() and 'entities' in tweet_data['extended_tweet'].keys() and 'media' in tweet_data['extended_tweet']['entities'].keys():
        count = 0
        for med in tweet_data['extended_tweet']['entities']['media']:
            if med['type'] == 'photo':
                count = count+1
        num_images = count
    
    if 'entities' in tweet_data.keys() and 'media' in tweet_data['entities'].keys():
        count = 0
        for med in tweet_data['entities']['media']:
            if med['type'] == 'photo':
                count = count+1
        num_images = count
    
    if num_images != 0:
        return num_images
    else:
        return None
        
def parse_json(filename):
    data = load_json(filename)
    
    data_dict = defaultdict(list)
    for i, tweet_data in enumerate(data['results']):
        if 'retweeted_status' in tweet_data.keys():
            tweet_data = tweet_data['retweeted_status']
            
        if 'extended_tweet' in tweet_data.keys():
            text = tweet_data['extended_tweet']['full_text']
        else:
            text = tweet_data['text']
        
        num_images = get_num_images(tweet_data)
        date_time = tweet_data['created_at']
        favorite_count = tweet_data['favorite_count']
        retweet_count = tweet_data['retweet_count']
        
        data_dict['text'].append(text)
        data_dict['date_time'].append(date_time)
        data_dict['num_favorites'].append(favorite_count)
        data_dict['num_retweets'].append(retweet_count)
        data_dict['num_images'].append(num_images)
    return data_dict

def draw_table(data_dict):
    table = Texttable()
    for i in range(len(data_dict['text'])):
        table.add_row([data_dict['text'][i], data_dict['date_time'][i], data_dict['num_favorites'][i], data_dict['num_retweets'][i], data_dict['num_images'][i]])
    print(table.draw())

if __name__ == '__main__':
    
    filename = 'data.json'
    data_dict = parse_json(filename)
    draw_table(data_dict)
    
#     print(pd.DataFrame(data_dict, dtype='int32'))

+-----------------------------+----------------------------+------+-----+------+
| @IEEEBigMM19 is also        | Wed Mar 20 08:19:24 +0000  | 1    | 1   | None |
| available on Facebook now.  | 2019                       |      |     |      |
| LIKE its Facebook page      |                            |      |     |      |
| https://t.co/B3Q0zmmzXb  to |                            |      |     |      |
| get the regular updates.    |                            |      |     |      |
| Check more details at       |                            |      |     |      |
| https://t.co/w9ZymoPisk     |                            |      |     |      |
|                             |                            |      |     |      |
| #IEEE #BigMM19 #Big         |                            |      |     |      |
| #Multimedia #Singapore      |                            |      |     |      |
+-----------------------------+----------------------------+------+-----+------+
| BigMM 2019 : IEEE BigMM   